In [3]:
!pip install pyacvd
!pip install tetgen
!pip install panel
!apt-get install -qq xvfb
!pip install pyvista panel -q
!pip install -q piglet pyvirtualdisplay

     |████████████████████████████████| 713 kB 4.6 MB/s eta 0:00:01


In [4]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(600, 400))
display.start()

In [5]:
import meshio
import os
import tetgen
import pyvista
import numpy as np
import pyacvd

In [6]:
# subdiv=6, num_cluster=2000
def refine_surface_mesh(mesh, subdiv=6, num_cluster=100):
    # First, refine surface mesh:
    clus = pyacvd.Clustering(mesh)
    clus.subdivide(subdiv)
    clus.cluster(num_cluster)
    remesh = clus.create_mesh()
    return remesh

In [7]:
def create_volume_mesh(mesh, mindihedral=20):
    # Next, convert surface mesh to volume mesh:
    tet = tetgen.TetGen(mesh)
    tet.tetrahedralize(order=1, mindihedral=mindihedral) 
    return tet.grid

In [8]:
def get_submesh(mesh, axis, cutoff):
    cell_center = mesh.cell_centers().points
    mask = cell_center[:, axis] < cutoff
    cell_ind = mask.nonzero()[0]
    submesh = mesh.extract_cells(cell_ind)
    return submesh

In [9]:
def visualise_mesh(grid, subgrid=None, title=None):
    pyvista.start_xvfb(wait=0.05)
    p = pyvista.Plotter(notebook=True, window_size=[960,480]) #
    title = "Breast Mesh" if title is None else title
    p.add_text(title, name="title", position="upper_edge")
    if subgrid is not None:
        p.add_mesh(grid, style="wireframe", color="k") 
        p.add_mesh(subgrid, lighting=True, show_edges=True) 
    else: 
        p.add_mesh(grid, show_edges=True, edge_color='k', lighting=False)
    
    p.show_axes()
    # p.show_bounds()
    p.show_grid()
    viewer = p.show(jupyter_backend='panel', return_viewer=True)
    return viewer

In [10]:
def transform_mesh(mesh):
    new_mesh = mesh.copy()
    new_mesh.rotate_x(90)
    new_mesh.rotate_z(90)
    # new_mesh.rotate_z(90)
    # new_mesh.rotate_y(180)
    new_mesh.scale(1000)
    min_pt = np.min(new_mesh.points, axis=0)
    new_mesh.translate(-1*min_pt)
    return new_mesh

In [11]:
mesh = pyvista.read('breast_closed.obj')
visualise_mesh(mesh, title='Original Surface Mesh')

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

In [20]:
mesh = transform_mesh(mesh)
visualise_mesh(mesh, title='Transformed Surface Mesh')

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

In [29]:
remesh_tet = refine_surface_mesh(mesh, subdiv=5, num_cluster=800)
pyvista.save_meshio('breast_tet.obj', remesh_tet)

In [30]:
remesh_tet = pyvista.read('breast_tet.obj')
visualise_mesh(remesh_tet)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

In [23]:
remesh_hex = refine_surface_mesh(mesh, subdiv=6, num_cluster=200)
pyvista.save_meshio('breast_hex.obj', remesh_hex)

In [26]:
remesh_hex = pyvista.read('breast_hex.obj')
visualise_mesh(remesh_hex)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

# Additional Visualisation

In [ ]:
remesh = remesh_tet

In [14]:
remesh = pyvista.read('breast_remeshed.obj')
visualise_mesh(remesh, title='Refined Surface Mesh')

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

In [12]:
transformed = transform_mesh(remesh)
visualise_mesh(transformed, title='Transformed Surface Mesh')

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

In [87]:
# Write surface meshes to new .obj files:
pyvista.save_meshio('breast.obj', transformed)

In [13]:
vol = create_volume_mesh(transformed, mindihedral=23)
subvol = get_submesh(vol, axis=1, cutoff=80)
visualise_mesh(vol, subvol, title='Volume Mesh with Cross-Section')

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)